In [14]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def fetch_binance_ohlc(symbol, start_date, end_date, interval='1d'):
    """
    Fetch OHLC data from Binance API for a given symbol and date range
    
    Parameters:
        symbol (str): Trading pair symbol (e.g., 'BTCUSDT')
        start_date (str/datetime): Start date in format 'YYYY-MM-DD' or datetime object
        end_date (str/datetime): End date in format 'YYYY-MM-DD' or datetime object
        interval (str): Kline interval (default '1d' - 1 day)
                       Options: '1m', '5m', '15m', '30m', '1h', '2h', '4h', 
                                '6h', '8h', '12h', '1d', '3d', '1w', '1M'
    
    Returns:
        pd.DataFrame: DataFrame with OHLC data and volume
    """
    # Convert dates to timestamp in milliseconds
    if isinstance(start_date, str):
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
    if isinstance(end_date, str):
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
    
    start_ts = int(start_date.timestamp() * 1000)
    end_ts = int(end_date.timestamp() * 1000)
    
    base_url = "https://api.binance.com/api/v3/klines"
    all_data = []
    
    current_start = start_ts
    max_records_per_request = 1000  # Binance's limit
    
    try:
        while current_start < end_ts:
            # Calculate end timestamp for this batch
            current_end = min(current_start + max_records_per_request * get_interval_ms(interval), end_ts)
            
            params = {
                'symbol': symbol,
                'interval': interval,
                'startTime': current_start,
                'endTime': current_end,
                'limit': max_records_per_request
            }
            
            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()
            
            if not data:
                break
                
            all_data.extend(data)
            
            # Move window forward
            current_start = int(data[-1][0]) + get_interval_ms(interval)
            
        # Process the data into a DataFrame
        columns = [
            'Open time', 'Open', 'High', 'Low', 'Close', 'Volume',
            'Close time', 'Quote asset volume', 'Number of trades',
            'Taker buy base volume', 'Taker buy quote volume', 'Ignore'
        ]
        
        df = pd.DataFrame(all_data, columns=columns)
        
        # Convert timestamp to datetime
        df['Open time'] = pd.to_datetime(df['Open time'], unit='ms')
        df['Close time'] = pd.to_datetime(df['Close time'], unit='ms')
        
        # Convert strings to numeric values
        numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
        df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, axis=1)
        
        # Filter only the relevant columns
        df = df[['Open time', 'Open', 'High', 'Low', 'Close', 'Volume']]
        
        return df
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching OHLC data: {e}")
        return pd.DataFrame()

def get_interval_ms(interval):
    """Convert interval string to milliseconds"""
    unit = interval[-1]
    value = int(interval[:-1])
    
    if unit == 'm':
        return value * 60 * 1000
    elif unit == 'h':
        return value * 60 * 60 * 1000
    elif unit == 'd':
        return value * 24 * 60 * 60 * 1000
    elif unit == 'w':
        return value * 7 * 24 * 60 * 60 * 1000
    elif unit == 'M':
        return value * 30 * 24 * 60 * 60 * 1000  # Approximate
    else:
        return 0

    

In [16]:
def calculate_rsi(data, periods=14):
    """
    Calculate the Relative Strength Index (RSI) for a given OHLC dataset.
    
    Parameters:
        data (pd.DataFrame): DataFrame with 'Close' prices.
        periods (int): Lookback period for RSI (default=14).
    
    Returns:
        pd.Series: RSI values.
    """
    # Calculate price changes
    delta = data['Close'].diff()
    
    # Separate gains and losses
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    # Calculate average gains and losses (SMA)
    avg_gain = gain.rolling(window=periods).mean()
    avg_loss = loss.rolling(window=periods).mean()
    
    # Calculate Relative Strength (RS)
    rs = avg_gain / avg_loss
    
    # Calculate RSI
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

In [18]:
def crossAbove(pre,current,threshold=30):
    if pre<threshold and current>threshold:
        return True
    else:
       return False

In [19]:
def crossBelow(pre,current,threshold=70):
    if pre>threshold and current<threshold:
        return True
    else:
       return False

In [24]:
symbol='BTCUSDT'
start_date='2025-07-04'
end_date='2025-07-14'
interval='15m'
# Fetch BTC/USDT daily data from Jan 1 to Dec 31, 2023
ohlc_data = fetch_binance_ohlc(
    symbol=symbol,
    start_date=start_date,
    end_date=end_date,
    interval=interval
)

In [25]:
# Assuming 'df' is your OHLC DataFrame with 'Close' column
ohlc_data['RSI'] = calculate_rsi(ohlc_data, periods=14)

In [26]:
ohlc_data['rsiCrossAbove'] = [False] * len(ohlc_data)
ohlc_data['rsiCrossBelow'] = [False] * len(ohlc_data)
for x in range(1,len(ohlc_data)):
    current = ohlc_data.iloc[x]["RSI"]
    pre=ohlc_data.iloc[x-1]["RSI"]
    ohlc_data.loc[x, 'rsiCrossAbove'] = crossAbove(pre,current)
    ohlc_data.loc[x, 'rsiCrossBelow'] = crossBelow(pre,current)

In [ ]:
outputFile=f'RSI CrossOver-{symbol}-{interval}-{start_date}-{end_date}.csv'
ohlc_data.to_csv(outputFile, index=False)  # index=False prevents writing row numbers

In [29]:
trades=ohlc_data[(ohlc_data['rsiCrossAbove']==True)|(ohlc_data['rsiCrossBelow']==True)]
tradesFile=f'Trades-RSI CrossOver-{symbol}-{interval}-{start_date}-{end_date}.csv'
trades.to_csv(tradesFile, index=False)  # index=False prevents writing row numbers